[과제] https://news.naver.com/ 사이트에서 4개의 뉴스 카테고리를 선택해서 뉴스기사를 크롤링 한 후 다음을 수행하세요.

- 뉴스기사 카테고리별 저장(파일 or DB)
- 텍스트 전처리 및 피처 벡터화
- 모델링 및 평가(성능개선 포함)

In [2]:
# 라이브러리 임포트
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import konlpy

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep

In [ ]:
# 드라이버 경로
path = 'C:/Users/kpo01/tool/chromedriver.exe'
# 브라우저 창을 띄우지 말고 수행하도록 옵션 지정
# options = webdriver.ChromeOptions()
# options.add_argument('headless') 
# driver = webdriver.Chrome(path, options=options)
driver = webdriver.Chrome(path)

news_df = pd.DataFrame(columns = ['text','category']) # text, category 컬럼으로 이루어진 df 생성
cat_list = ['101', '102', '103', '105'] # 경제, 사회, 생활/문화, IT/과학

for cat in cat_list:
    for page in range(1,25):
        url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={cat}#&date=%2000:00:00&page={page}'
        driver.get(url)
        print('현재위치: ', driver.current_url) # 진행 현황 확인

        # 헤드라인 기사 제목 클릭해서 기사 본문으로 들어가기
        for i in range(1,5): 
            sleep(3)
            for j in range(1,6): 
                try: 
                    print(i, j)
                    driver.find_element(By.XPATH, f'//*[@id="section_body"]/ul[{i}]/li[{j}]/dl/dt[2]/a').click()
                    sleep(0.7)
                    # 본문 수집(개행문자 제거)
                    text = driver.find_element(By.XPATH, '//*[@id="articleBodyContents"]').text.replace('\n', '')
                    # 데이터프레임에 본문과 카테고리를 행으로 삽입
                    news_df.loc[len(news_df)] = [text, cat]
                    # 뒤로 가기
                    driver.back()
                    sleep(0.7)
                except:
                    print(i, j)
                    driver.find_element(By.XPATH, f'//*[@id="section_body"]/ul[{i}]/li[{j}]/dl/dt/a').click()
                    sleep(0.7)
                    # 본문 수집(개행문자 제거)
                    text = driver.find_element(By.XPATH, '//*[@id="articleBodyContents"]').text.replace('\n', '')
                    # 데이터프레임에 본문과 카테고리를 행으로 삽입
                    news_df.loc[len(news_df)] = [text, cat]
                    # 뒤로 가기
                    driver.back()
                    sleep(0.7)
            

display(news_df)

# CSV파일로 저장하기
news_df.to_csv('./news.csv')

In [107]:
import pandas as pd
news_df = pd.read_csv('./news.csv',index_col=0)

In [108]:
news_df.category.value_counts()

102    480
103    480
105    480
101    479
Name: category, dtype: int64

In [109]:
from sklearn.model_selection import train_test_split
X = news_df['text']
y = news_df['category']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [102]:
from konlpy.tag import Okt
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

okt = Okt()

def okt_tokenizer(text):
    tokens_ko = okt.morphs(text)
    return tokens_ko

tfidf_vect = TfidfVectorizer(tokenizer = okt_tokenizer, ngram_range = (1, 2), min_df = 3, max_df = 0.9)


In [104]:
tfidf_vect.fit(X_train)
tfidf_matrix_train = tfidf_vect.transform(X_train)

In [105]:
lg_clf = LogisticRegression(random_state=0)

params = {'C':[10,10.5,12]}

grid_cv = GridSearchCV(lg_clf,param_grid=params,cv=3,scoring='accuracy')
grid_cv.fit(tfidf_matrix_train,y_train)
print(grid_cv.best_params_,round(grid_cv.best_score_,4))

{'C': 10.5} 0.8163


In [106]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer를 이용해 테스트 데이터를 TF-IDF값으로 피처 변환함.
tfidf_matrix_test = tfidf_vect.transform(X_test)

# Classifier는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
preds = grid_cv.best_estimator_.predict(tfidf_matrix_test)

print(f'Logistic Regression 정확도: {accuracy_score(y_test,preds)}')

Logistic Regression 정확도: 0.8489583333333334
